In [1]:
# 1609492611000, 1641028611000, 1672564611000 (2021, 2022, 2023) 1677632461000

In [2]:
# ! conda install -c conda-forge ta --yes

In [13]:
import requests
import pandas as pd
import pandas_ta as p_ta
import ta
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import time

In [4]:
from backtesting import Backtest
from backtesting import Strategy
from backtesting.lib import crossover

/Users/gd01/opt/anaconda3/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [5]:
interval_arr = ['15m', '1h', '4h', '1d']
start_time_arr = [1677632461000]

In [6]:
timezone = 8
symbol = 'ethusdt'
symbol_C = symbol.upper()
interval = '15m'

start_time = 1679274061000
end_time = round(time.time() * 1000)

# step between timestamps in milliseconds
step = 60000 * 3600

In [7]:
def create_raw(symbol, interval_arr, start_time, end_time, step):
    
    url = "https://api.binance.com/api/v3/klines"
    
    for interval in interval_arr:

        raw_df = pd.DataFrame()
        
        for timestamp in range(start_time, end_time, step):
            params = {"symbol": symbol_C,
                      "interval": interval,
                      "startTime": timestamp,
                      "endTime": timestamp + step}
            response = requests.get(url, params=params).json()
            out = pd.DataFrame(response, columns = ["Open time", "Open", "High", "Low", "Close",
                                                   "Volume", "Close_Time", "Quote asset volume",
                                                   "Number of trades", "Taker buy base asset volume",
                                                   "Taker buy quote asset volume", "Ignore"])
            raw_df = pd.concat([raw_df, out], axis = 0)

        raw_df = raw_df[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']]

        raw_df.to_hdf(f'klines_{symbol}_{interval}.h5', key='df', mode='w')
        print(f"Created {symbol}_{interval}")

In [8]:
loop_start_time = time.time()
create_raw(symbol, interval_arr, min(start_time_arr), end_time, step)
loop_end_time = time.time()
print("Time taken to execute for loop:", loop_end_time - loop_start_time, "seconds")

Created ethusdt_15m
Created ethusdt_1h
Created ethusdt_4h
Created ethusdt_1d
Time taken to execute for loop: 7.4510509967803955 seconds


In [9]:
h5 = pd.read_hdf(f'klines_{symbol}_{interval_arr[-1]}.h5', key='df')
print(h5.tail(1))

      Close_Time           Open          Close           High            Low  \
0  1681257599999  1910.21000000  1916.06000000  1937.37000000  1906.25000000   

            Volume  
0  163583.63610000  


In [10]:
def time_format(timezone):
#     df['Close_Time'] = pd.to_datetime(df['Close_Time'], unit='ms') + pd.Timedelta(hours=timezone)
#     df['Close_Time'] = df['Close_Time'].dt.strftime('%Y-%m-%d %H:%M:%S')
    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)
#     df.index = df.index.strftime('%Y-%m-%d %H:%M:%S')

In [11]:
klines_cache = {}

def get_klines(symbol, interval, start_time, end_time):
    if (symbol, interval) not in klines_cache:
        klines_cache[(symbol, interval)] = pd.read_hdf(f'klines_{symbol}_{interval}.h5', key='df')

    df = klines_cache[(symbol, interval)].query(f"Close_Time >= {start_time} and Close_Time <= {end_time}")

    df = df[['Close_Time', 'Open', 'Close', "High", "Low", 'Volume']].astype(float)

    df = df.set_index('Close_Time')
#     df = df.reset_index(drop=True)

    df.index = pd.to_datetime(df.index, unit='ms') + pd.Timedelta(hours=timezone)


    return df

In [38]:
cash = 200
leverage = 10
commission = 0.05/100

In [41]:
class RsiOscillator(Strategy):
    
    rsi_up = 70
    rsi_low = 30
    rsi_window = 14
    
    kd_int = 14
    d_int = 3
    
    ema1 = 8
    ema2 = 18
    ema3 = 38

    # Do as much initial computation as possible
    def init(self):
        
        self.rsi = self.I(p_ta.rsi, pd.Series(self.data.Close), self.rsi_window)

        self.ema1 = self.I(ta.trend.ema_indicator, pd.Series(self.data.Close), window=self.ema1)  
        self.ema2 = self.I(ta.trend.ema_indicator, pd.Series(self.data.Close), window=self.ema2)   
        self.ema3 = self.I(ta.trend.ema_indicator, pd.Series(self.data.Close), window=self.ema3)   

        
        kd = ta.momentum.StochasticOscillator(high = pd.Series(self.data.High),
                                              low = pd.Series(self.data.Low),
                                              close = pd.Series(self.data.Close),
                                              window = self.kd_int,
                                              smooth_window = self.d_int)
        
        self.k = self.I(kd.stoch)
        self.d = self.I(kd.stoch_signal)

    # Step through bars one by one

    def next(self):
        if self.rsi[-5:-1].any() > self.rsi_up and crossover(self.d, self.k):
            self.position.close()
        elif self.rsi[-1:-1].any() < self.rsi_low and crossover(self.k, self.d):
            self.buy()

bt = Backtest(get_klines(symbol, interval, start_time, end_time),
              RsiOscillator,
              cash = cash,
              commission = commission,
              margin = 1/leverage)

stats = bt.run()
print(stats)
bt.plot(filename = 'backtesting.html')

/var/folders/9s/k__n74kn5976jstw3vsw95600000gn/T/ipykernel_23398/2858022007.py:41: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(get_klines(symbol, interval, start_time, end_time),
/var/folders/9s/k__n74kn5976jstw3vsw95600000gn/T/ipykernel_23398/2858022007.py:49: UserWarning: 'Can't superimpose OHLC data with rule 'None'(index datetime resolution: 'microsecond'). Skipping.
  bt.plot(filename = 'backtesting.html')


Start                     2023-03-20 09:14...
End                       2023-04-11 16:59...
Duration                     22 days 07:45:00
Exposure Time [%]                   98.083216
Equity Final [$]                    338.82128
Equity Peak [$]                     354.48128
Return [%]                           69.41064
Buy & Hold Return [%]                7.965448
Return (Ann.) [%]              1504909.779137
Volatility (Ann.) [%]        272361959.086233
Sharpe Ratio                         0.005525
Sortino Ratio                     8485.547855
Calmar Ratio                     28089.326725
Max. Drawdown [%]                  -53.575858
Avg. Drawdown [%]                  -14.000755
Max. Drawdown Duration        7 days 22:45:00
Avg. Drawdown Duration        1 days 10:50:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       7.806278
Worst Trade [%]                      7.806278
Avg. Trade [%]                    

Row(id='14406', ...)